In [7]:
#import os
#os.chdir('../')

In [9]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

from src import db_connect

import hydra
from omegaconf import DictConfig

In [11]:
@hydra.main(config_path="../config", config_name="main")
def create_graph_langs(cfg: DictConfig):

    # Retrieve database config path using Hydra's cfg object
    config_data = get_database_config_from_json(cfg.db.credentials)

    # Connect to the database
    conn = connect_to_db(config_data)

create_graph_langs()

C:\Users\Alice\AppData\Local\Temp\ipykernel_13648\3944813816.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="../config", config_name="main")
usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: argument --shell-completion/-sc: ignored explicit argument '9031'


AttributeError: 'tuple' object has no attribute 'tb_frame'